## Analyze functional effects on HA-mediated entry into MDCK-SIAT1 cells

In [4]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# read in data
func_data = pd.read_csv('../results/func_effects/averages/MDCKSIAT1_entry_func_effects.csv')
func_data.head()

,site,wildtype,mutant,effect,effect_std,times_seen,n_selections
0,1,Q,*,-4.9450,0.0000,16.25,4
1,1,Q,A,-0.1226,0.2296,7.50,4
2,1,Q,C,-0.5732,0.5667,5.75,4
3,1,Q,D,0.2550,0.3448,6.50,4
4,1,Q,E,0.2941,0.0502,9.00,4


In [6]:
# read in 60y entropy of sites
entropy_df = pd.concat(
    [pd.read_csv(
        'data/nextstrain_groups_neherlab_flu_seasonal_h3n2_ha1_60y_diversity.tsv', sep = '\t'
    ),
    pd.read_csv(
        'data/nextstrain_groups_neherlab_flu_seasonal_h3n2_ha2_60y_diversity.tsv', sep = '\t'
    ).assign(position=lambda x: x['position'] + 329)]
).rename(
    columns={'position': 'site'}
).drop(columns=['gene'])

entropy_df.head()

,site,entropy
0,1,0.011
1,2,0.291
2,3,0.741
3,4,0.002
4,5,0.141


In [7]:
# read in structure mapping
site_map = pd.read_csv('../data/site_numbering_map.csv')
site_map.head()

,sequential_site,reference_site,sequential_wt,region,rbs_region
0,1,1,Q,HA1,outside RBS
1,2,2,K,HA1,outside RBS
2,3,3,I,HA1,outside RBS
3,4,4,P,HA1,outside RBS
4,5,5,G,HA1,outside RBS


In [8]:
# annotate the data with epitope information
func_data_ann = pd.merge(
    pd.merge(
        func_data, 
        site_map[['reference_site', 'region', 'rbs_region']], 
        left_on='site', 
        right_on='reference_site', 
        how='right'
    ).drop(
        columns=['reference_site']
    ).query(
        'mutant not in ["*", "-"]' # don't want stop codons/indels
    ).query(
        'mutant != wildtype'
    ).query(
        'times_seen >= 2'
    ).query(
        'effect_std <= 2'
    ),
    entropy_df,
    on='site',
    how='left'
)

# sites with 0 entropy were not included
func_data_ann['entropy'] = func_data_ann['entropy'].fillna(0)
func_data_ann.head()

,site,wildtype,mutant,effect,effect_std,times_seen,n_selections,region,rbs_region,entropy
0,1,Q,A,-0.1226,0.2296,7.50,4,HA1,outside RBS,0.011
1,1,Q,C,-0.5732,0.5667,5.75,4,HA1,outside RBS,0.011
2,1,Q,D,0.2550,0.3448,6.50,4,HA1,outside RBS,0.011
3,1,Q,E,0.2941,0.0502,9.00,4,HA1,outside RBS,0.011
4,1,Q,F,-0.7141,0.6042,7.00,4,HA1,outside RBS,0.011


### Distribution of functional effects across classic receptor binding pocket regions

In [9]:
# Define custom colors for each RBS region
colors = {
    'outside RBS': '#bdbebb',
    'RBS 130-loop': '#725663',
    'RBS 150-loop': '#725663',
    'RBS 190-loop': '#725663',
    'RBS 220-loop': '#725663',
    'RBS base': '#725663'
}

# Create the jitter plot layer
points = alt.Chart(func_data_ann).mark_circle(size=40, opacity=0.3, stroke=None, strokeWidth=0).encode(
    y=alt.Y(
        "rbs_region",
        axis=alt.Axis(
            grid=False,
            titleFontSize=16,
            titleFontWeight='normal',
            labelFontSize=16,
            labelFontWeight='normal',
        )
    ).title(None),
    x=alt.X(
        'effect',
        title=(["Mutation effect on cell entry"]),
        axis=alt.Axis(
            grid=False,
             tickCount=4,
            titleFontSize=16,
            titleFontWeight='normal',
            labelFontSize=16,
            labelFontWeight='normal',
        )
    ),
    yOffset="jitter:Q",
    color=alt.Color(
            "rbs_region",
            scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())),
            legend=None
    ),
    tooltip=['site', 'wildtype', 'mutant', 'effect'],
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

# Add black median line
median_line = alt.Chart(func_data_ann).mark_tick(
    color='black',
    thickness=3,
    size=25
).encode(
    y='rbs_region:N',
    x='median(effect):Q'
)

vline = alt.Chart().mark_rule(
    color='black',
    size=1.25,
    opacity=1.0,
    strokeDash=[6,6]
).encode(x=alt.X(datum=0))

# Combine layers with median line on top
chart = alt.layer(
    points,
    median_line,
    vline
).properties(
    height=200, 
    width=200
)

mean_entropy_rbs = func_data_ann[['site', 'rbs_region', 'entropy']].drop_duplicates().groupby(
    "rbs_region"
)["entropy"].mean().reset_index().rename(
    columns={'entropy': 'mean_entropy'}
)

bar_chart = alt.Chart(mean_entropy_rbs).mark_bar(size=20).encode(
    x=alt.X(
        'mean_entropy:Q',
        title='Mean entropy',
        axis=alt.Axis(
            grid=False,
            tickCount=2,
            titleFontSize=16,
            titleFontWeight='normal',
            labelFontSize=16,
            labelFontWeight='normal',
        )
    ),
    y=alt.Y(
        'rbs_region:N',
        title=None,
        axis=None
    ),
    color=alt.Color(
            "rbs_region",
            scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())),
            legend=None
    ),
    tooltip=['rbs_region', 'mean_entropy']
).properties(
    height=200,
    width=80
)

# Concatenate the plots
combined_chart = alt.concat(
    chart,  # jitter + median line + vline chart
    bar_chart, # entropy bar chart
    spacing=10
).resolve_scale(
    y='shared'
).properties(
    title=alt.TitleParams(
        text='Receptor binding pocket region',
        anchor='middle',
        fontSize=16,
        fontWeight='bold',
    )
)

combined_chart.display()

alt.ConcatChart(...)

### Distribution of functional effects across classic antigenic regions

In [11]:
# Define custom colors for each RBS region
colors = {
    'epitope-A': '#FFB547',
    'epitope-B': '#FFB547',
    'epitope-C': '#FFB547',
    'epitope-D': '#FFB547',
    'epitope-E': '#FFB547',
    'HA1': '#bdbebb',
    'HA2': '#767676',
}

order = ['epitope-A', 'epitope-B', 'epitope-C', 'epitope-D', 'epitope-E', 'HA1', 'HA2']

# Create the jitter plot layer
points = alt.Chart(func_data_ann).mark_circle(size=40, opacity=0.3, stroke=None, strokeWidth=0).encode(
    y=alt.Y(
        "region",
        axis=alt.Axis(
            grid=False,
            titleFontSize=16,
            titleFontWeight='normal',
            labelFontSize=16,
            labelFontWeight='normal',
        ),
        sort=order
    ).title(None),
    x=alt.X(
        'effect',
        title=(["Mutation effect on cell entry"]),
        axis=alt.Axis(
            grid=False,
             tickCount=4,
            titleFontSize=16,
            titleFontWeight='normal',
            labelFontSize=16,
            labelFontWeight='normal',
        )
    ),
    yOffset="jitter:Q",
    color=alt.Color(
            "region",
            scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())),
            legend=None
    ),
    tooltip=['site', 'wildtype', 'mutant', 'effect'],
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

# Add black median line
median_line = alt.Chart(func_data_ann).mark_tick(
    color='black',
    thickness=3,
    size=25
).encode(
    y=alt.Y('region:N', sort=order),
    x='median(effect):Q'
)

vline = alt.Chart().mark_rule(
    color='black',
    size=1.25,
    opacity=1.0,
    strokeDash=[6,6]
).encode(x=alt.X(datum=0))

# Combine layers with median line on top
chart = alt.layer(
    points,
    median_line,
    vline
).properties(
    height=200, 
    width=200
)

mean_entropy_epitope = func_data_ann[['site', 'region', 'entropy']].drop_duplicates().groupby(
    "region"
)["entropy"].mean().reset_index().rename(
    columns={'entropy': 'mean_entropy'}
)

bar_chart = alt.Chart(mean_entropy_epitope).mark_bar(size=20).encode(
    x=alt.X(
        'mean_entropy:Q',
        title='Mean entropy',
        axis=alt.Axis(
            grid=False,
            tickCount=2,
            titleFontSize=16,
            titleFontWeight='normal',
            labelFontSize=16,
            labelFontWeight='normal',
        )
    ),
    y=alt.Y(
        'region:N',
        sort=order, 
        title=None,
        axis=None
    ),
    color=alt.Color(
            "region",
            scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())),
            legend=None
    ),
    tooltip=['region', 'mean_entropy']
).properties(
    height=200,
    width=80
)

# Concatenate the plots
combined_chart = alt.concat(
    chart,  # jitter + median line + vline chart
    bar_chart, # entropy bar chart
    spacing=10
).resolve_scale(
    y='shared'
).properties(
    title=alt.TitleParams(
        text='Antigenic region',
        anchor='middle',
        fontSize=16,
        fontWeight='bold',
    )
)

combined_chart.display()

alt.ConcatChart(...)